In [1]:
#Import librerie di cui necessitiamo
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
"""Creiamo una directory in cui andremo a salvare tutte le immagini che man mano
verranno generate in fase di training del modello per valutare quanto il modello
sta funzionando bene"""
os.makedirs("results_images", exist_ok=True)

In [3]:
"""Sfruttando la libreria argparse creiamo una sorta di namespace contenente
tutte le variabili che ci serviranno da ora in avanti"""
parser = argparse.ArgumentParser()
parser.add_argument('-f')
parser.add_argument("--n_epochs", type=int, default=600, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=4, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--n_classes", type=int, default=10, help="number of classes for dataset")
parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=3, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=10, help="interval between image sampling")
opt = parser.parse_args()
print(opt)

Namespace(b1=0.5, b2=0.999, batch_size=64, channels=3, f='/root/.local/share/jupyter/runtime/kernel-15bebdef-58a4-4f1c-a236-df7ae226268e.json', img_size=32, latent_dim=100, lr=0.0002, n_classes=10, n_cpu=4, n_epochs=600, sample_interval=10)


In [4]:
"""Al reference cuda assegniamo True se la GPU è disponibile, altrimenti gli
assegniamo False"""
cuda = True if torch.cuda.is_available() else False

In [5]:
"""Tale funzione come parametro riceve un modulo presente all'interno del
generatore o del discriminatore e se il modulo è di tipo Conv o sottotipo di
Conv (ad esempio Conv2d) allora vengono inizializzati i pesi di tale modulo, se
il modulo è di tipo BatchNorm2d allora vengono inizializzati i pesi e i bias di
tale modulo"""
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [6]:
"""Modello del generatore"""
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.latent_dim)

        self.fcl = nn.Linear(opt.latent_dim, 128 * (opt.img_size // 4) * (opt.img_size // 4))

        self.conv_layers = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),

            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),

            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, kernel_size=3, padding=1),

            nn.Tanh()
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        fcl_out = self.fcl(gen_input)
        img = fcl_out.view(fcl_out.size(0), 128, opt.img_size // 4, opt.img_size // 4)
        imgs = self.conv_layers(img)
        return imgs

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def block(in_channels, out_channels, bn=True):
            layers = nn.ModuleList([nn.Conv2d(in_channels, out_channels, 3, 2, 1),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Dropout2d(0.2)])
            if bn:
                layers.append(nn.BatchNorm2d(out_channels, 0.8))
            return layers

        self.conv_layers = nn.Sequential(
            *block(opt.channels, 16, bn=False),
            *block(16, 32),
            *block(32, 64),
            *block(64, 128)
        )

        self.adv_layer = nn.Sequential(
            nn.Linear(128 * 2 * 2, 1),
            nn.Sigmoid()
        )


        self.aux_layer = nn.Sequential(
            nn.Linear(128 * 2 * 2, opt.n_classes),
            nn.Softmax()
        )


    def forward(self, img):
        img = self.conv_layers(img)
        fcl_in = img.view(img.size(0), -1)
        validity = self.adv_layer(fcl_in)
        label = self.aux_layer(fcl_in)
        return validity, label

In [8]:
#Definiamo le Loss Function che ci serviranno"""
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

In [9]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [10]:
"""Se la GPU è disponibile spostiamo i modelli e le Loss all'interno della GPU"""
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

In [11]:
#Inizializziamo i parametri dei moduli presenti all'interno del generatore e del discriminatore
"""Supponiamo di considerare la funzione apply invocata su generator. Tale funzione
in maniera iterativa prende ogni singolo modulo presente all'interno di generator
e lo passa come parametro della funzione weights_init_normal"""
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (conv_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.2, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.2, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.2, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_la

In [12]:
# Configure data loader
os.makedirs("../../data/CIFAR10", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "../../data/CIFAR10",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)


Extracting ../../data/CIFAR10/cifar-10-python.tar.gz to ../../data/CIFAR10


In [13]:
#Creiamo gli ottimizzatori
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [14]:
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [16]:
def sample_image(n_row, epoch):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    """Per generare z, ossia il minibatch contenente rumore casuale da mandare in ingresso
    al generatore abbiamo creato un Tensor 2-D avente un numero di righe pari a
    n_row**2 e un numero di colonne pari a 100. Dopodiché, dal momento che vogliamo
    che questo Tensor contenga sempre gli stessi valori anche se la funzione sample_image
    viene chiamata più volte abbiamo deciso di wrappare tale Tensor all'interno di Variable.
    Così facendo, anche se la funzione sample_image viene chiamata più volte, il valore di z
    rimane invariato"""
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    """Per generare labels, ossia il minibatch contenente i target da mandare in ingresso
    al generatore abbiamo creato un Tensor 1-D avente un numero di elementi pari a 100
    (numero di righe di z), dove l'elemento di indice 0 assume 0, l'elemento di indice 1 assume 1, l'elemento di indice 2 assume 2
    l'elemento di indice 9 assume 9, l'elemento di indice 10 assume 0, l'elemento di indice 11 assume 1 e così via.
    Inoltre, per far si che il valore di labels non cambi nonostante la funzione sample_image venga chiamata più volte abbiamo wrappato
    tale Tensor all'interno di Variable"""
    """Prendiamo il Tensor 2-D z e il Tensor 1-D labels e mandiamoli in ingresso al generatore"""
    gen_imgs = generator(z, labels)
    """Il numero di immmagini che vengono prodotte in uscita dal gneratore è pari a n_row**2"""
    save_image(gen_imgs.data, "results_images/Epoca%d.png" % epoch, nrow=n_row, normalize=True)
    """Tramite save_image all'interno della directory results_images che abbiamo creato
    precedentemente inseriamo una griglia costituita dalle n_row**2 immagini prodotte in uscita
    dal generatore disposte su n_row righe"""

In [17]:
for epoch in range(opt.n_epochs):
  history_d_minibatch_accuracy = []
  history_d_minibatch_loss = []
  history_g_minibatch_loss = []
  for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0] #64

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        """valid è un Tensor 2-D avente un numero di righe pari a batch_size(64)
        e un numero di colonne pari a 1. Tale Tensor contiene tutti 1 e ci 
        serve per calcolare l'adversarial loss per le immagini reali
        """
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)
        """fake è un Tensor 2-D avente un numero di righe pari a batch_size(64)
        e un numero di colonne pari a 1. Tale Tensor contiene tutti 0 e ci 
        serve per calcolare l'adversarial loss per le immagini fake
        """

        # Configure input
        """Invocando il metodo type su un Tensor è possibile effettuare il
        casting di tale Tensor"""
        real_imgs = Variable(imgs.type(FloatTensor))
        """real_imgs è un minibatch contenente un numero di immagini reali pari
        a minibatch_size(64)"""
        labels = Variable(labels.type(LongTensor))
        """labels è un minibatch contenente i minibatch_size(64) target
        corrispondenti alle 64 immagini reali presenti nel minibatch real_imgs"""

        # -----------------
        #  Train Generator
        # -----------------
        """Resettiamo il gradiente"""
        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        """z è un minibatch contenente 64 rumori casuali, ognuno di 100 elementi"""
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        """gen_labels è un Tensor 1-D contenente 64 (batch_size) target, uno per
        ogni singolo rumore casuale presente nel minibatch z"""
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)
        """gen_imgs è un minibatch contenente le 64 immagini (batch_size)
        generate dal generatore"""
        # Loss measures generator's ability to fool the discriminator
        """A questo punto prendiamo il minibatch contenente le 64 immagini
        generate dal generatore e le mandiamo in ingresso al discriminatore"""
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))
        """Calcoliamo il gradiente della g_loss"""
        g_loss.backward()
        """Ottimizziamo i parametri del generatore"""
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        """Resettiamo il gradiente"""
        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        """gen_imgs contiene le 64 (batch_size) immagini generate dal generatore
        durante la fase di training del generatore"""

        """Utilizzando detach() nel momento in cui attuiamo la backpropagation
        per aggiornare i parametri del discriminatore quello che accade è che
        effettivamente vengono aggiornati solo ed esclusivamente i parametri del
        discriminatore e non anche quelli del generatore.
        Se non usassimo detach() nel momento in cui attuaiamo la backpropagation
        per aggiornare i parametri del discriminatore, oltre ai parametri del
        discriminatore, verrebbero aggiornati anche quelli del generatore e questo
        non va bene in quanto vogliamo aggiornare solo i parametri del
        discriminatore"""
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2
        """Calcoliamo il gradiente della loss del discriminatore"""
        d_loss.backward()
        """Ottimizziamo i parametri del discriminatore"""
        optimizer_D.step()


        #Calcoliamo l'accuratezza del discriminatore nel classificare correttamente la classe di appartenenza di ogni singola immagine (fake o reale) che riceve
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        """pred ha dimensione 128x10 in quanto è dato dalla concatenazione per riga
        di real_aux (Tensor 2-D di dimensione 64x10 contenente per ogni immagine reale fornita al discriminatore
        le probabilità con cui l'immagine appartiene ad ognuna delle 10 classi) e di
        fake_aux (Tensor 2-D di dimensione 64x10 contenente per ogni immagine fake fornita al discriminatore
        le probabilità con cui l'immagine appartiene ad ognuna delle 10 classi)"""
        target = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        """target ha dimensione 128 in quanto è dato dalla concatenazione per riga
        di labels (Tensor 1-D di dimensione 64 contenente per ogni immagine reale fornita al discriminatore il relativo target)
        e di gen_labels (Tensor 1-D di dimensione 64 contenente per ogni immagine fake fornita al discriminatore il relativo target)
        """
        d_minibatch_accuracy = np.mean(np.argmax(pred, axis=1) == target)
        history_d_minibatch_accuracy.append(d_minibatch_accuracy)
        history_d_minibatch_loss.append(d_loss.item())
        history_g_minibatch_loss.append(g_loss.item())

        if (epoch+1) % opt.sample_interval == 0:
            sample_image(n_row=10, epoch=epoch+1)

  d_epoch_accuracy = np.mean(history_d_minibatch_accuracy)
  d_epoch_loss = np.mean(history_d_minibatch_loss)
  g_epoch_loss = np.mean(history_g_minibatch_loss)
  print(f"Epoch {epoch+1}: Discriminator_Loss={d_epoch_loss:.4f}, Discriminator_Accuracy={d_epoch_accuracy*100:.4f}, Generator_Loss={g_epoch_loss:.4f}")


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 1: Discriminator_Loss=1.4342, Discriminator_Accuracy=25.8202, Generator_Loss=1.4198
Epoch 2: Discriminator_Loss=1.2722, Discriminator_Accuracy=59.5189, Generator_Loss=1.1660
Epoch 3: Discriminator_Loss=1.2613, Discriminator_Accuracy=62.2083, Generator_Loss=1.1323
Epoch 4: Discriminator_Loss=1.2566, Discriminator_Accuracy=63.1664, Generator_Loss=1.1211
Epoch 5: Discriminator_Loss=1.2523, Discriminator_Accuracy=63.9976, Generator_Loss=1.1200
Epoch 6: Discriminator_Loss=1.2501, Discriminator_Accuracy=64.4851, Generator_Loss=1.1172
Epoch 7: Discriminator_Loss=1.2463, Discriminator_Accuracy=65.1495, Generator_Loss=1.1160
Epoch 8: Discriminator_Loss=1.2441, Discriminator_Accuracy=65.7049, Generator_Loss=1.1128
Epoch 9: Discriminator_Loss=1.2417, Discriminator_Accuracy=66.2704, Generator_Loss=1.1127
Epoch 10: Discriminator_Loss=1.2392, Discriminator_Accuracy=66.7469, Generator_Loss=1.1101
Epoch 11: Discriminator_Loss=1.2376, Discriminator_Accuracy=67.0027, Generator_Loss=1.1109
Epoch 12